## Data-Oriented and ASyncIO

Reference

- https://towardsdatascience.com/data-oriented-programming-with-python-ef478c43a874/
- https://medium.com/velotio-perspectives/an-introduction-to-asynchronous-programming-in-python-af0189a88bbb
- https://realpython.com/python-async-features/